In [ ]:
from platform import python_version
print(python_version())

### Calc best cutoffs
### Simulate many indexs
### Calc if random genes can be equal or better than best cutoffs

## Definitions:
  - LFC table has:
    - abs_LFC (the absolute LFC cutoff value)
    - FDR_LFC, its FDR or p-value adjusted
  - The enriched pathway table has:
    - FDR_pathway cutoff value
   
### Default values for LFC table:
  - abs_LFC=1
  - FDR_LFC=0.05
  - therefore, a DEG/DAP is abs(LFC) >= 1 and FDR < 0.05

### Default values for Enriched Pathways:
  - FDR_pathway=0.05
  - therefore, an enriched pathways has FDR < 0.05 and at least 3 DEGs/DAPs

### Calculating the best cutoffs:
  - We proposed and calculated many indexes to define a new statistics to flebilize the LFC and Enriched Pathway cutoffs.
    - Indexes are calculated for each case, each cutoff, and each resulting enriched pathway.
  - To find the possible best LFC/FDR expression and FDR pathway cutoffs:
     - We look for a high number in n_pathway and n_DEGs_in_pathway, having a low FDR_LFC and a high absLFC.
       - The default FDR_LFC (0.05):
          - It may have fewer DEGs, resulting in fewer enriched pathways.
          - It may have fewer enriched pathways, even having many DEGs/DAPs.
       - Therefore, a trade-off exists between optimizing (abs_LFC and FDR_LFC cutoffs) and (FDR_pathway cutoffs, n_pathways, and n_DEGs_in_pathways.)

### TOI calculates the trade-off between "LFC" and "Enriched Pathways" cufoffs

  - We proposed and calculated four Trade-Off Indexes (toi):

<p style="font-size: 20px; color: yellow;">
$toi1=\sqrt{-log{_{10}}{FDR_{pathway}} * \frac{n}{N} }$ </p>

<p style="font-size: 20px; color: cyan;">
$toi2=\sqrt{-log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} }$ </p>

<p style="font-size: 20px; color: orange;">
$toi3=(-log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} * \frac{n}{N})^{1/3}$ </p>

<p style="font-size: 20px; color: pink;">
$toi4=(abs\_LFC * -log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} * \frac{n}{N})^{1/4}$ </p>

where,
  - n is the number of DEGs/DAPs found in the pathway
  - N is the total number of annotated DEGs/DAPs in the pathway (depend in the database, our default database is Reactome 2022)

### Then we searched for the best cutoffs
  - In each 5 percentile of the index histogram, we look for the best abs_LFC, FDR_LFC, FDR_pathway:
  -  We expected that the best cutoff should be in the right tail of the histogram (high index value.)
  -  High index values must have a high number of n_pathways and n DEGs in pathways.

### Testing the best cutoffs (for each case)

  - Is the new set of cutoffs correct? good enough?
  - How to establish that the calculated cutoff is correct?
  - To answer these questions we calculated the chi-square test between the "best cutoff" and the "default"
    - Best cutoff has:
      - n DEGs/DAPs in pathways
      - n DEGs/DAPs not in pathways
    - The Default cutoff may have:
      - n DEGs/DAPs in pathways
      - n DEGs/DAPs not in pathways
      - The DEGs/DAPs can be:
        - greater or equal number of the best cutoff DEGs/DAPs
        - fewer number of the best cutoff DEGs/DAPs:
           - in this case, one complements the number of DEGs/DAPs with random genes not DEGs/DAPs (found in the experiment)

#### Chi-square test:

DEGs/DAPs | # in pathway | no in pathway
--- | --- | --- 
 Best cutoff |     A      |   B  
 Default cutoff |   C | D 

Chi-square p-value:
  - p-value < 0.05 denotes that both distributions are not similar; therefore, random genes could not reach the best cutoff DEGs/DAPs; in conclusion, the best cutoff was not found randomly.
  - p-value \>= 0.05 denotes that both distributions are similar, and the best cutoff can be achieved randomly.

In [ ]:
import os, sys, pickle

import numpy as np
import pandas as pd
pd.set_option('display.width', 100)
pd.set_option('max_colwidth', 80)
pd.set_option("display.precision", 3)

import yaml

import seaborn as sns
sns.set_context("notebook", font_scale=1.4)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

sys.path.insert(1, '../src/')

from Basic import *
from enricher_lib import *
from config_lib import *

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

# !pip3 install pyyaml
with open('params.yml', 'r') as file:
    dic_yml=yaml.safe_load(file)

In [ ]:
root0=dic_yml['root0']
email=dic_yml['email']

project=dic_yml['project']
s_project=dic_yml['s_project']

gene_protein=dic_yml['gene_protein']
s_omics=dic_yml['s_omics']

has_age=dic_yml['has_age']
has_gender=dic_yml['has_gender']

want_normalized=dic_yml['want_normalized']

abs_lfc_cutoff_inf=dic_yml['abs_lfc_cutoff_inf']
s_pathw_enrichm_method=dic_yml['s_pathw_enrichm_method']
num_min_degs_for_ptw_enr=dic_yml['num_min_degs_for_ptw_enr']

tolerance_pathway_index=dic_yml['tolerance_pathway_index']
type_sat_ptw_index=dic_yml['type_sat_ptw_index']
saturation_lfc_index=dic_yml['saturation_lfc_index']
chosen_model_sampling=dic_yml['chosen_model_sampling']

case_list=dic_yml['case_list']

pval_pathway_cutoff=dic_yml['pval_pathway_cutoff']
fdr_pathway_cutoff=dic_yml['fdr_pathway_cutoff']
num_of_genes_cutoff=dic_yml['num_of_genes_cutoff']

run_list=dic_yml['run_list']
chosen_model_list=dic_yml['chosen_model_list']
i_dfp_list=dic_yml['i_dfp_list']

exp_normalization='quantile_norm' if want_normalized else None
normalization='not_normalized' if exp_normalization is None else exp_normalization

cfg=Config(project, s_project, case_list, root0)

case=case_list[0]

n_genes_annot_ptw, n_degs, n_degs_in_ptw, n_degs_not_in_ptw, degs_in_all_ratio=-1,-1,-1,-1,-1
abs_lfc_cutoff, fdr_lfc_cutoff, n_degs, n_degs_up, n_degs_dw=cfg.get_best_lfc_cutoff(case, 'not_normalized')


print(f"G/P LFC cutoffs: lfc={abs_lfc_cutoff:.3f}; fdr={fdr_lfc_cutoff:.3f}")
print(f"Pathway cutoffs: pval={pval_pathway_cutoff:.3f}; fdr={fdr_pathway_cutoff:.3f}; num of genes={num_of_genes_cutoff}")

In [ ]:
enr=enricheR(gene_protein, s_omics, project, s_project, root0,
             case_list, has_age, has_gender, clone_objects=False,
             exp_normalization=exp_normalization, geneset_num=0, 
             num_min_degs_for_ptw_enr=num_min_degs_for_ptw_enr, 
             tolerance_pathway_index=tolerance_pathway_index, 
             s_pathw_enrichm_method=s_pathw_enrichm_method,
             abs_lfc_cutoff_inf=abs_lfc_cutoff_inf, 
             type_sat_ptw_index=type_sat_ptw_index, saturation_lfc_index=saturation_lfc_index)

case=case_list[0]

enr.cfg.set_default_best_lfc_cutoff(normalization, abs_lfc_cutoff=1, fdr_lfc_cutoff=0.05)
ret, degs, degs_ensembl, dfdegs=enr.open_case(case, verbose=False)
print("\nEcho Parameters:")
enr.echo_parameters()
geneset_num=enr.geneset_num

In [ ]:
enr.case, enr.group, enr.gender, enr.age, enr.geneset_num, enr.abs_lfc_cutoff_inf

In [ ]:
enr.abs_lfc_cutoff_inf, abs_lfc_cutoff_inf

In [ ]:
for case in case_list:
    ret, degs, degs_ensembl, dfdegs=enr.open_case(case, save_file=True, verbose=False)
    enr.echo_parameters()
    print("\n\n\n")

In [ ]:
print(">>>", case)

ret, _, _, _ = enr.open_case(case, verbose=False)
enr.pathway_pval_cutoff, enr.pathway_fdr_cutoff

In [ ]:
verbose=True
ret_lfc = enr.open_dflfc_ori(verbose=verbose)
ret_lfc

In [ ]:
save_enriched_ptws_excel_odt=False
verbose=True
force=False

ret_enr = enr.open_enrichment_analysis(force=force, save_enriched_ptws_excel_odt=save_enriched_ptws_excel_odt, verbose=verbose)
ret_enr

### Find another case=WNT

In [ ]:
case=case_list[1]
ret, degs, degs_ensembl, dfdegs=enr.open_case(case, verbose=False)
enr.echo_parameters()

In [ ]:
enr.case, enr.group, enr.gender, enr.age, enr.geneset_num, enr.abs_lfc_cutoff_inf

### Reference database

In [ ]:
enr.geneset_num, enr.geneset_lib, enr.dbs_list

In [ ]:
enr.set_db(0, verbose=True)

In [ ]:
fname_final_ori, fname_ori, title=enr.set_lfc_names()
fname_final_ori, title

### Sampling cutoffs

In [ ]:
want=False
geneset_num_list=[0, 1, 2, 4, 5, 7]

if want:
    for geneset_num in geneset_num_list:
        enr.set_db(geneset_num, verbose=True)
    
        s_start=f"enricher_{enr.geneset_lib}"
    
        for case in case_list:
            files=[x for x in os.listdir(enr.root_enrichment) if x.startswith(s_start) and case in x]
            print("\tcase", case, len(files))
    
        print('')

### Define case

In [ ]:
geneset_num=0
enr.set_db(geneset_num, verbose=True)
enr.geneset_num, enr.geneset_lib

In [ ]:
i=0
case=case_list[i]

ret, degs, degs_ensembl, dfdegs=enr.open_case(case, verbose=False)
print(f"G/P cutoff: lfc={enr.abs_lfc_cutoff:.3f}; lfc_fdr={enr.fdr_lfc_cutoff:.3f}")
len(degs), len(degs_ensembl)

In [ ]:
i=1
case=case_list[i]

ret, degs, degs_ensembl, dfdegs=enr.open_case(case, verbose=False)
print(f"G/P cutoff: lfc={enr.abs_lfc_cutoff:.3f}; lfc_fdr={enr.fdr_lfc_cutoff:.3f}")
len(degs), len(degs_ensembl)

### best_cutoff_quantiles() - for Reactome - return dfcut

In [ ]:
fname=enr.fname_enr_gene_stat%(enr.case, enr.geneset_lib, enr.normalization)
filefull=os.path.join(enr.root_ressum, fname)

os.path.exists(filefull), fname

In [ ]:
enr.set_enrichment_name()

In [ ]:
force=False
verbose=False

for case in case_list:
    print(">>>", case)
    _=enr.calc_enrichment_cutoff_params_and_ndxs_per_case_and_geneset_lib(case, force=force, verbose=verbose)


In [ ]:
verbose=True
case=case_list[0]

dfi=enr.calc_enrichment_cutoff_params_and_ndxs_per_case_and_geneset_lib(case, force=False, verbose=verbose)
print(len(dfi))
dfi.head(3)

### best_cutoff_quantiles() - return dfcut

In [ ]:
fname=enr.fname_enr_gene_stat%(enr.case, enr.geneset_lib, enr.normalization)
filefull=os.path.join(enr.root_ressum, fname)

os.path.exists(filefull), fname

In [ ]:
enr.set_enrichment_name()

In [ ]:
verbose=True

case=case_list[1]
print(">>>", case)
dfi=enr.calc_enrichment_cutoff_params_and_ndxs_per_case_and_geneset_lib(case, force=False, verbose=verbose)
print(len(dfi))
dfi.head(3)

## build_all_cutoffs_table(col)
  - loop case_list
    - best_cutoff_quantiles()
      - calc_enrichment_cutoff_params_and_ndxs_per_case_and_geneset_lib

## Next, calc_best_cutoffs_params()

In [ ]:
enr.open_enriched_pathways_summary()

In [ ]:
enr.abs_lfc_cutoff_inf

#### build_all_cutoffs_table

  - for cols=['toi1_median', 'toi2_median', 'toi3_median', 'toi4_median']
    - build_all_cutoffs_table()
      - for each case
        - for  med_max_ptw in ['median', 'maximum', 'pathway']:
          - best_cutoff_quantiles
            - dfi=enr.calc_enrichment_cutoff_params_and_ndxs_per_case_and_geneset_lib(case, force=...)


In [ ]:
cols=['toi1_median', 'toi2_median', 'toi3_median', 'toi4_median']
geneset_num=0
enr.set_db(geneset_num)
print(enr.geneset_num, enr.geneset_lib, '\n')

force=False
verbose=False

for col in cols:
    print(">>>", col)
    dfcut=enr.build_all_cutoffs_table(selected_toi_col=col, force=force, verbose=False)

In [ ]:
dfcut.columns

In [ ]:
verbose=True

case=case_list[0]
print(">>>", case)
dfi=enr.calc_enrichment_cutoff_params_and_ndxs_per_case_and_geneset_lib(case, force=False, verbose=verbose)

maxi  =dfi.toi4_median.max()
median=dfi.toi4_median.median()
maxi, median

In [ ]:
dfi.case.unique()

In [ ]:
verbose=True

case=case_list[1]
print(">>>", case)

dfi=enr.calc_enrichment_cutoff_params_and_ndxs_per_case_and_geneset_lib(case, force=False, verbose=False)

maxi  =dfi.toi4_median.max()
median=dfi.toi4_median.median()
maxi, median

In [ ]:
df3=dfi[ (dfi.toi4_median == maxi)]
df3

In [ ]:
cols=['case', 'geneset_num', 'normalization', 'med_max_ptw', 'parameter', 'quantile',
      'quantile_val', 'quantile_val_inf', 'quantile_val_sup', 
      'abs_lfc_cutoff', 'fdr_lfc_cutoff', 
      'pathway_pval_cutoff', 'pathway_fdr_cutoff', 'num_of_genes_cutoff',
      'n_pathways', 'n_degs_in_pathways', 
      'n_degs_in_pathways_mean', 'n_degs_in_pathways_median', 'n_degs_in_pathways_std', 
      'toi1_mean', 'toi1_median', 'toi1_std',
      'toi2_mean', 'toi2_median', 'toi2_std',
      'toi3_mean', 'toi3_median', 'toi3_std',
      'toi4_mean', 'toi4_median', 'toi4_std',]


cols=['case', 'med_max_ptw', 'quantile', 'toi4_median', 'abs_lfc_cutoff', 'fdr_lfc_cutoff',
        'pathway_pval_cutoff', 'pathway_fdr_cutoff', 'n_pathways', 'n_degs_in_pathways',
        'toi1_median', 'toi2_median', 'toi3_median']

### Look for different approaches (sorting)

In [ ]:
dfcut.case.unique()

In [ ]:
case=case_list[0]
case

In [ ]:
col='toi1_median'
dfcut=enr.build_all_cutoffs_table(selected_toi_col=col, force=False, verbose=False)
df2=dfcut[(dfcut.case == case) & (dfcut.n_degs_in_pathways > 3) & (dfcut.med_max_ptw == 'median')]
df2=df2.sort_values(col, ascending=False)
df2[cols].head(6)

In [ ]:
col='toi2_median'
dfcut=enr.build_all_cutoffs_table(selected_toi_col=col, force=False, verbose=False)
df2=dfcut[(dfcut.case == case) & (dfcut.n_degs_in_pathways > 3) & (dfcut.med_max_ptw == 'median')]
df2=df2.sort_values(col, ascending=False)
df2[cols].head(6)

In [ ]:
col='toi3_median'
dfcut=enr.build_all_cutoffs_table(selected_toi_col=col, force=False, verbose=False)
df2=dfcut[(dfcut.case == case) & (dfcut.n_degs_in_pathways > 3) & (dfcut.med_max_ptw == 'median')]
df2=df2.sort_values(col, ascending=False)
df2[cols].head(6)

In [ ]:
col='toi4_median'
dfcut=enr.build_all_cutoffs_table(selected_toi_col=col, force=False, verbose=False)
df2=dfcut[(dfcut.case == case) & (dfcut.n_degs_in_pathways > 3) & (dfcut.med_max_ptw == 'median')]
df2=df2.sort_values(col, ascending=False)
df2[cols].head(6)

### Bad way to cut - by n_pathways

In [ ]:
dfa=dfcut[dfcut.case == case].sort_values(['n_pathways', 'n_degs_in_pathways', 'pathway_fdr_cutoff'], ascending=[False, False, True])
dfa[cols].head(9)

In [ ]:
dfa=dfcut[dfcut.case == case].sort_values(['n_degs_in_pathways', 'n_pathways', 'pathway_fdr_cutoff'], ascending=[False, False, True])
dfa[cols].head(9)

In [ ]:
dfa=dfcut[dfcut.case == case].sort_values(['pathway_fdr_cutoff', 'n_pathways', 'n_degs_in_pathways'], ascending=[True, False, False])
dfa[cols].head(9)

In [ ]:
dfa=dfcut[dfcut.case == case].sort_values(['pathway_fdr_cutoff', 'n_pathways', 'n_degs_in_pathways'], ascending=[True, False, False])
dfa[cols].head(9)

### Revisting DEGs and n_Pathwys x abs_LFC cutoff correlation

#### return dic_fig
  - which in ['deg', 'up', 'down']

In [ ]:
verbose=False
plot_up_down=False

corr_cutoff=-0.90
nregs_fdr=5

for case in case_list:
    print(">>", case)
    ret, dic_fig, df_fdr=enr.plot_nDEG_curve_per_LFC_FDR(case, width=1100, height=700, title=None, 
                                                           corr_cutoff=corr_cutoff, nregs_fdr=nregs_fdr, verbose=verbose)
    
    print("deg")
    dic_fig['deg'].show()

    if plot_up_down:
        print("up")
        dic_fig['up'].show()
        print("down")
        dic_fig['down'].show()
        
    print()

### Revisting DEGs and n_Pathways x toi4 median

In [ ]:
colors=['navy', 'red', 'darkcyan', 'darkgreen', 'orange', 'brown', 'darksalmon',
        'magenta', 'darkturquoise', 'orange', 'darkred', 'indigo', 'magenta', 'maroon', 'black',
        'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'olivedrab', 'navy'] + plotly_colors_proteins

In [ ]:
verbose=False
selected_toi_col='toi4_median'

dfcut=enr.build_all_cutoffs_table(selected_toi_col=selected_toi_col, force=False, verbose=verbose)
dfcut.columns

maxi_x=np.round(dfcut[selected_toi_col].max(), 3) + 0.001
maxi_x

In [ ]:
case=enr.case
print(">>>", case)

df_fdr=enr.open_fdr_lfc_correlation(case=case)
print(len(df_fdr))
df_fdr.head(3)

In [ ]:
fdr=0.20

dfcut=enr.build_all_cutoffs_table(selected_toi_col, force=False, verbose=False)
df2=dfcut[ (dfcut.case == case) & (dfcut.fdr_lfc_cutoff == fdr)  & (dfcut.med_max_ptw == 'median')]
df2

In [ ]:
enr.abs_lfc_cutoff_inf, case

In [ ]:
dfcut=enr.build_all_cutoffs_table(selected_toi_col, force=False, verbose=False)
# print(dfcut.columns)
cols=['case', 'toi4_median', 'med_max_ptw', 
       'quantile', 'quantile_val_inf', 'quantile_val_sup',
        'abs_lfc_cutoff', 'fdr_lfc_cutoff', 'pathway_fdr_cutoff',  'n_pathways',
       'n_degs_in_pathways']  #  'toi1_median', 'toi2_median',  'toi3_median',
case='WNT'

df2=dfcut[ (dfcut.case == case) & (dfcut.normalization == enr.normalization) & (dfcut.geneset_num == enr.geneset_num) &
             (dfcut.med_max_ptw == 'median')] 
df2[cols]

In [ ]:
df_all_fdr=enr.open_all_fdr_lfc_correlation()

### The whole TOI landdscape

In [ ]:
selected_toi_col

In [ ]:
height=1200

fig=enr.plot_degs_in_pathways_vs_toi_per_case(selected_toi_col=selected_toi_col, title=None, plot_all_dfi=True,
                                                width=1100, height=height, sel_colors=None, plot_bgcolor='lightgray', verbose=False)

if fig: fig.show()

In [ ]:
fig=enr.plot_degs_in_pathways_vs_toi_per_case(selected_toi_col=selected_toi_col, title=None, plot_all_dfi=False,
                                              width=1100, height=1000, sel_colors=None, plot_bgcolor='lightgray', verbose=False)

if fig: fig.show()

### best_cutoff_quantiles() - for Reactome - return dfcut

In [ ]:
fname, fname_cut=enr.set_enrichment_name()
# name='enricher_Reactome_2022_medulloblastoma_microarray_for_WNT_x_ctrl_not_normalized_cutoff_lfc_0.950_fdr_0.200_pathway_pval_0.050_fdr_0.450_num_genes_3.tsv'
"; ".join(fname_cut.split('_'))

In [ ]:
cols=['case', 'med_max_ptw', 'quantile', 'toi4_median', 'abs_lfc_cutoff', 'fdr_lfc_cutoff',
      'pathway_pval_cutoff', 'pathway_fdr_cutoff', 'n_pathways', 'n_degs_in_pathways',
      'toi1_median', 'toi2_median', 'toi3_median']

In [ ]:
case=case_list[0]
selected_toi_col='toi4_median'

df2=dfcut[(dfcut.case == case) & (dfcut.med_max_ptw == 'median')]
df2=df2.sort_values(selected_toi_col, ascending=False)
df2[cols].head(6)

## Calc best cutoffs and save in config

### Comparing: toi4 (or 1,2,3), n_pathways, n_degs_in_pathways
### which is them best?

  - n_best_sample=4 ... the lowest FDRs and highest abs_LFC

In [ ]:
enr.geneset_num, enr.geneset_lib, enr.normalization

In [ ]:
dfc=enr.build_all_cutoffs_table('toi4_median')
dfc[dfc.case == 'G4'].head(30)

In [ ]:
cols2=['case', 'med_max_ptw', 'quantile', 'toi4_median', 'toi4_mean', 'abs_lfc_cutoff', 'fdr_lfc_cutoff', 'pathway_fdr_cutoff', 
       'n_pathways', 'n_degs_in_pathways', 'n_degs_in_pathways_mean',
       'n_degs_in_pathways_median', 'n_degs_in_pathways_std',
       'toi1_median', 'toi2_median', 'toi3_median']

save_config=False
dfconfig=enr.calc_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample=1, save_config=False, verbose=True)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfconfig=enr.calc_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample=2, save_config=False, verbose=save_config)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfconfig=enr.calc_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample=3, save_config=False, verbose=save_config)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfconfig=enr.calc_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample=4, save_config=False, verbose=save_config)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfconfig=enr.calc_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample=5, save_config=save_config, verbose=save_config)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfconfig=enr.calc_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample=6, save_config=False, verbose=save_config)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

### Multiple best points

In [ ]:
dfpoints=enr.display_best_cutoff_params(npoints=10, selected_toi_col='toi4_median', med_max_ptw='median')
dfpoints

### Turn to True to save config
  - G4 ipoint 7
  -  WNT ipoint 4

In [ ]:
save_config=False

n_best_sample_list=[4, 7]
dfconfig=enr.calc_multiple_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample_list=n_best_sample_list, save_config=save_config, verbose=save_config)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfbest=enr.cfg.dfbest_cutoffs[cols2]
dfbest[dfbest.med_max_ptw == 'median']

### Chosen columns

In [ ]:
dfcut.columns

In [ ]:
cols1=['case', 'med_max_ptw', 'quantile', 'toi4_median']
cols2=['abs_lfc_cutoff', 'fdr_lfc_cutoff', 'pathway_fdr_cutoff',
        'n_pathways', 'n_degs_in_pathways']

### Comparing: toi4 (or 1,2,3), n_pathways, n_degs_in_pathways

In [ ]:
case='WNT'

In [ ]:
col='n_pathways'
nrows=6
dfcut=enr.build_all_cutoffs_table(col, force=False, verbose=False)
case=case_list[0]
dfa=dfcut[ (dfcut.case == case) & (dfcut.med_max_ptw == 'median') ][cols]
dfa=dfa.sort_values(['n_pathways', 'n_degs_in_pathways'], ascending=[False, False])
dfa[cols1+cols2].head(nrows)

In [ ]:
col='toi1_median'
n_rows=6
dfcut=enr.build_all_cutoffs_table(col, force=False, verbose=False)
dfa=dfcut[ (dfcut.case == case) & (dfcut.med_max_ptw == 'median') ]
dfa=dfa.sort_values(col, ascending=False)
dfa[cols1+[col]+cols2].head(n_rows)

In [ ]:
col='toi2_median'
dfcut=enr.build_all_cutoffs_table(col, force=False, verbose=False)
dfa=dfcut[ (dfcut.case == case) & (dfcut.med_max_ptw == 'median')]
dfa=dfa.sort_values(col, ascending=False)
dfa[cols1+[col]+cols2].head(n_rows)

In [ ]:
col='toi3_median'
dfcut=enr.build_all_cutoffs_table(col, force=False, verbose=False)
dfa=dfcut[ (dfcut.case == case) & (dfcut.med_max_ptw == 'median') ]
dfa=dfa.sort_values(col, ascending=False)
dfa[cols1+[col]+cols2].head(n_rows)

In [ ]:
col='toi4_median'
dfcut=enr.build_all_cutoffs_table(col, force=False, verbose=False)
dfa=dfcut[ (dfcut.case == case) & (dfcut.med_max_ptw == 'median') ]
dfa=dfa.sort_values(col, ascending=False)
dfa[cols1+cols2].head(n_rows)

### TOI4

In [ ]:
selected_toi_col='toi4_median'

fig_list=enr.plot_genes_and_pathways_frequecies_per_cases(selected_toi_col,  width=1100, height=700)

print(">>>", selected_toi_col)
print(f"# {enr.s_deg_dap}s")
fig_list[0].show()
print("# n pathways")
fig_list[1].show()

### Selected best cutoffs per case

In [ ]:
dfconfig=enr.calc_multiple_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample_list=n_best_sample_list, save_config=False, verbose=False)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfbest=enr.cfg.open_best_ptw_cutoff(verbose=False)
dfbest=dfbest[dfbest.med_max_ptw == 'median']
print(len(dfbest))
dfbest[cols]

### n_pathways: to confirm, is it working?

In [ ]:
fig_list=enr.plot_genes_and_pathways_frequecies_per_cases('n_pathways',  width=1100, height=700)

fig_list[0].show()
print("")
fig_list[1].show()

## Why toi4_median is the best approach?

#### balance between best LFC cutoffs and Pathway cutoffs

In [ ]:
n_best_sample_list

In [ ]:
dfconfig=enr.calc_multiple_best_cutoffs_params(selected_toi_col='n_pathways', n_best_sample_list=n_best_sample_list, save_config=False, verbose=save_config)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfconfig=enr.calc_multiple_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample_list=n_best_sample_list, save_config=False, verbose=save_config)
dfconfig=dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

## It tries to minimize fdr and maximize abs_lfc cutoffs!!!
### look for toi4_median in both tables

### Open best cutoffs (config)

In [ ]:
dfbest=enr.cfg.open_best_ptw_cutoff(verbose=False)
dfbest2=dfbest[dfbest.med_max_ptw == 'median']
dfbest2[cols]

### Saving DEGs

In [ ]:
for case in case_list:
    ret, degs, degs_ensembl, dfdegs=enr.open_case(case, save_file=True, verbose=False)
    enr.echo_parameters()
    print("\n\n\n")

### At least 10 DEGs for each case

In [ ]:
verbose=False
per_biotype= True
ensembl=False

dfa=enr.summary_degs_up_down(per_biotype=per_biotype, ensembl=ensembl, verbose=verbose)
print(len(dfa))
cols=list(dfa.columns)[1:]
dfa[cols]=dfa[cols].astype(int)
dfa

In [ ]:
title=f'Up and Down {enr.s_deg_dap}s with the best cutoff'
fig, dfa=enr.barplot_up_down_genes_per_case(title=title, width=1100, height=700, verbose=False)
fig.show()

### Summary DEGs + Up and Down

In [ ]:
verbose=False
per_biotype= False
ensembl=False

dfa=enr.summary_degs_up_down(per_biotype=per_biotype, ensembl=ensembl, verbose=verbose)
print(len(dfa))
dfa

In [ ]:
dfbest=enr.cfg.open_best_ptw_cutoff(verbose=True)
dfbest2=dfbest[dfbest.med_max_ptw == 'median']
dfbest2

In [ ]:
dfsum=enr.summary_degs_and_pathways(force=False, verbose=False)
cols5=list(dfsum.columns)
dfsum